<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/ChatHaruhi3_messages%E7%BB%84%E8%A3%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

02/10

- [x] 在foo embedding的前提下调通embedding组装，考虑没有rag
- [x] 考虑单rag
- [x] 考虑复合rag
- [x] 考虑rag with另一个句子的retrieve
- [x] 接入tiktoken
- [x] 接入bge

02/11

- [x] 替换为直接载入

In [1]:
!pip install -q datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 396 (delta 260), reused 270 (delta 164), pack-reused 3
Receiving objects: 100% (396/396), 2.92 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (260/260), done.
/content/Zero-Haruhi


In [3]:
from ChatHaruhi import ChatHaruhi

In [4]:
def get_response( message ):
    return "语言模型输出了角色扮演的结果"

chatbot = ChatHaruhi( role_name = '小猫咪', persona = "你扮演一只小猫咪", llm = get_response )


In [5]:
chatbot.chat(user = "用户", text = "你好")

query_rags: [] rest_limit = 1784


'语言模型输出了角色扮演的结果'

In [6]:
chatbot.chat(user = "用户2", text = "你好")

new user 用户2 included in conversation
query_rags: [] rest_limit = 1784


'语言模型输出了角色扮演的结果'

## 对没有rag的persona进行测试

In [7]:
chatbot = ChatHaruhi( role_name = '小猫咪', persona = "你扮演一只小猫咪", llm = get_response )
message = chatbot.get_message( user = "用户", text = "你好" )
for msg in message:
    print(f"{msg['role']}\n{msg['content']}\n\n")

query_rags: [] rest_limit = 1784
system
You are now in roleplay conversation mode. Pretend to be 小猫咪 whose persona follows:
你扮演一只小猫咪

You will stay in-character whenever possible, and generate responses as if you were 小猫咪


user
你好




In [9]:
import time
# 开始计时
start_time = time.time()

chatbot.verbose = False

# 执行一百次模拟函数
for _ in range(100):
    message = chatbot.get_message( user = "用户", text = "你好" )

# 结束计时
end_time = time.time()

# 计算总耗时
print( end_time - start_time )

0.03575563430786133


## 对有随机rag下进行测试

In [10]:
persona = """你扮演爸爸

{{RAG对话}}
"""

role_name = "爸爸"

stories = ["爸爸的爸爸是爷爷", "爸爸的妈妈是奶奶"]

chatbot = ChatHaruhi( role_name = role_name, persona = persona, llm = get_response ,\
                     stories = stories)

message = chatbot.get_message( user = "用户", text = "你好" )
for msg in message:
    print(f"{msg['role']}\n{msg['content']}\n\n")

re-extract vector for 2 stories


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

100%|██████████| 2/2 [00:07<00:00,  3.90s/it]

query_rags: [{'n': 1, 'max_token': -1, 'query': '你好', 'lid': 2}] rest_limit = 1782
system
You are now in roleplay conversation mode. Pretend to be 爸爸 whose persona follows:
你扮演爸爸

###
爸爸的妈妈是奶奶


You will stay in-character whenever possible, and generate responses as if you were 爸爸


user
你好




## 对rag要求数量超过stories下进行测试

In [11]:
persona = """你扮演爸爸

{{RAG对话}}
{{RAG对话}}
{{RAG对话}}
"""

role_name = "爸爸"

stories = ["爸爸的爸爸是爷爷", "爸爸的妈妈是奶奶"]

chatbot = ChatHaruhi( role_name = role_name, persona = persona, llm = get_response ,\
                     stories = stories)

message = chatbot.get_message( user = "用户", text = "你好" )
for msg in message:
    print(f"{msg['role']}\n{msg['content']}\n\n")

re-extract vector for 2 stories


100%|██████████| 2/2 [00:00<00:00, 52.90it/s]

query_rags: [{'n': 1, 'max_token': -1, 'query': '你好', 'lid': 2}, {'n': 1, 'max_token': -1, 'query': '你好', 'lid': 3}, {'n': 1, 'max_token': -1, 'query': '你好', 'lid': 4}] rest_limit = 1770
system
You are now in roleplay conversation mode. Pretend to be 爸爸 whose persona follows:
你扮演爸爸

###
爸爸的妈妈是奶奶
###
爸爸的爸爸是爷爷



You will stay in-character whenever possible, and generate responses as if you were 爸爸


user
你好




## 100个stories的测试

In [12]:
persona = """你扮演爸爸

{{RAG对话}}
{{RAG对话|高兴}}
{{RAG多对话|token<=1500|n<=5}}
"""

role_name = "爸爸"

stories = []

for i in range(100):
    story = str(i+1) + "爸爸的爸爸是爷爷\n"
    stories.append(story)


chatbot = ChatHaruhi( role_name = role_name, persona = persona, llm = get_response ,\
                     stories = stories)

message = chatbot.get_message( user = "用户", text = "你好" )
for msg in message:
    print(f"{msg['role']}\n{msg['content']}\n\n")

re-extract vector for 100 stories


100%|██████████| 100/100 [00:01<00:00, 51.36it/s]


query_rags: [{'n': 1, 'max_token': -1, 'query': '你好', 'lid': 2}, {'n': 1, 'max_token': -1, 'query': '高兴', 'lid': 3}, {'n': 5, 'max_token': 1500, 'query': '你好', 'lid': 4}] rest_limit = 1756
system
You are now in roleplay conversation mode. Pretend to be 爸爸 whose persona follows:
你扮演爸爸

###
97爸爸的爸爸是爷爷
###
68爸爸的爸爸是爷爷
###
29爸爸的爸爸是爷爷
###
49爸爸的爸爸是爷爷
###
28爸爸的爸爸是爷爷
###
95爸爸的爸爸是爷爷
###
19爸爸的爸爸是爷爷


You will stay in-character whenever possible, and generate responses as if you were 爸爸


user
你好




## 照理说openai的也可以，我来试一试

因为foo会随机一个1536的vector

In [14]:
chatbot = ChatHaruhi( role_name = "haruhi", llm = get_response , max_story_n_haruhi = 2)

message = chatbot.get_message( user = "用户", text = "你好" )

query_rags: [{'n': 2, 'max_token': 1000, 'query': '你好', 'lid': 8}] rest_limit = 1518


In [15]:
chatbot.verbose = False
message = chatbot.get_message( user = "用户", text = "你好" )

In [16]:
import time

# 开始计时
start_time = time.time()

# 执行一百次模拟函数
for _ in range(100):
    message = chatbot.get_message( user = "用户", text = "你好" )

# 结束计时
end_time = time.time()

# 计算总耗时
print( end_time - start_time )

5.341772794723511


接下来就是要给所有RolePlaying的角色增加bge的列

In [18]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device.type == 'cpu')

True


---

下面是过往的代码存档，don't run

In [ ]:
from ChatHaruhi.NaiveDB import NaiveDB
from ChatHaruhi.embeddings import foo_embedding, foo_bge_zh_15, foo_bce, foo_openai, embedname2columnname, get_bge_zh_embedding
from ChatHaruhi.utils import base64_to_float_array, base64_to_string, tiktoken_counter
from ChatHaruhi.sugar_map import sugar_role_names, enname2zhname

In [ ]:
def get_text_from_data( data ):
    if "text" in data:
        return data['text']
    elif "enc_text" in data:
        # from .utils import base64_to_string
        return base64_to_string( data['enc_text'] )
    else:
        print("warning! failed to get text from data ", data)
        return ""

def parse_rag(text):
    lines = text.split("\n")
    ans = []

    for i, line in enumerate(lines):
        if "{{RAG对话}}" in line:
            ans.append({"n": 1, "max_token": -1, "query": "default", "lid": i})
        elif "{{RAG对话|" in line:
            query_info = line.split("|")[1].rstrip("}}")
            ans.append({"n": 1, "max_token": -1, "query": query_info, "lid": i})
        elif "{{RAG多对话|" in line:
            parts = line.split("|")
            max_token = int(parts[1].split("<=")[1])
            max_n = int(parts[2].split("<=")[1].rstrip("}}"))
            ans.append({"n": max_n, "max_token": max_token, "query": "default", "lid": i})

    return ans

class ChatHaruhi:
    def __init__(self,
                 role_name = None,
                 user_name = None,
                 persona = None,
                 stories = None,
                 story_vecs = None,
                 role_from_hf = None,
                 role_from_jsonl = None,
                 llm = None, # 默认的message2response的函数
                 llm_async = None, # 默认的message2response的async函数
                 user_name_in_message = "default",
                 verbose = None,
                 embed_name = None,
                 embedding = None,
                 db = None,
                 token_counter = "default",
                 max_input_token = 1800,
                 max_len_story_haruhi = 1000,
                 max_story_n_haruhi = 5
                 ):

        self.verbose = True if verbose is None or verbose else False

        self.db = db

        self.embed_name = embed_name

        self.max_len_story_haruhi = max_len_story_haruhi # 这个设置只对过往Haruhi的sugar角色有效
        self.max_story_n_haruhi = max_story_n_haruhi # 这个设置只对过往Haruhi的sugar角色有效

        self.last_msg = None

        if embedding is None:
            self.embedding = self.set_embedding_with_name( embed_name )

        if persona and role_name and stories and story_vecs and len(stories) == len(story_vecs):
            # 完全从外部设置，这个时候要求story_vecs和embedding的返回长度一致
            self.persona, self.role_name, self.user_name = persona, role_name, user_name
            self.build_db(stories, story_vecs)
        elif persona and role_name and stories:
            # 从stories中提取story_vecs，重新用self.embedding进行embedding
            story_vecs = self.extract_story_vecs(stories)
            self.persona, self.role_name, self.user_name = persona, role_name, user_name
            self.build_db(stories, story_vecs)
        elif role_from_hf:
            # 从hf加载role
            self.persona, new_role_name, self.stories, self.story_vecs = self.load_role_from_hf(role_from_hf)
            if new_role_name:
                self.role_name = new_role_name
            else:
                self.role_name = role_name
            self.user_name = user_name
            self.build_db(self.stories, self.story_vecs)
        elif role_from_jsonl:
            # 从jsonl加载role
            self.persona, new_role_name, self.stories, self.story_vecs = self.load_role_from_jsonl(role_from_jsonl)
            if new_role_name:
                self.role_name = new_role_name
            else:
                self.role_name = role_name
            self.user_name = user_name
            self.build_db(self.stories, self.story_vecs)
        elif persona and role_name:
            # 这个时候也就是说没有任何的RAG，
            self.persona, self.role_name, self.user_name = persona, role_name, user_name
            self.db = None
        elif role_name and self.check_sugar( role_name ):
            # 这个时候是sugar的role
            self.persona, self.role_name, self.stories, self.story_vecs = self.load_role_from_sugar( role_name )
            self.build_db(self.stories, self.story_vecs)
            self.add_rag_prompt_after_persona()
        else:
            raise ValueError("persona和role_name必须同时设置，或者role_name是ChatHaruhi的预设人物")

        self.llm, self.llm_async = llm, llm_async
        if not self.llm and self.verbose:
            print("warning, llm没有设置，仅get_message起作用，调用chat将回复idle message")

        self.user_name_in_message = user_name_in_message
        self.previous_user_pool = set([user_name]) if user_name else set()
        self.current_user_name_in_message = user_name_in_message.lower() == "add"

        self.idle_message = "idel message, you see this because self.llm has not been set."

        if token_counter.lower() == "default":
            # TODO change load from util
            self.token_counter = tiktoken_counter
        elif token_counter == None:
            self.token_counter = lambda x: 0
        else:
            self.token_counter = token_counter
            if self.verbose:
                print("user set costomized token_counter")

        self.max_input_token = max_input_token

        self.history = []

    def check_sugar(self, role_name):
        return role_name in sugar_role_names

    def load_role_from_sugar(self, role_name):
        en_role_name = sugar_role_names[role_name]
        new_role_name = enname2zhname[en_role_name]
        role_from_hf = "silk-road/ChatHaruhi-RolePlaying/" + en_role_name
        persona, _, stories, story_vecs = self.load_role_from_hf(role_from_hf)

        return persona, new_role_name, stories, story_vecs

    def add_rag_prompt_after_persona( self ):
        rag_sentence = "{{RAG多对话|token<=" + str(self.max_len_story_haruhi) + "|n<=" + str(self.max_story_n_haruhi) + "}}"
        self.persona += "Classic scenes for the role are as follows:\n" + rag_sentence

    def set_embedding_with_name(self, embed_name):
        if embed_name is None:
            self.embed_name = "bge_zh"
            return get_bge_zh_embedding
        elif embed_name == "foo":
            return foo_embedding
        elif embed_name == "bge_zh":
            return foo_bge_zh_15
        elif embed_name == "bce":
            return foo_bce
        elif embed_name == "openai" or embed_name == "luotuo_openai":
            return foo_openai

    def set_new_user(self, user):
        if len(self.previous_user_pool) > 0 and user not in self.previous_user_pool:
            if self.user_name_in_message.lower() == "default":
                if self.verbose:
                    print(f'new user {user} included in conversation')
                self.current_user_name_in_message = True
        self.user_name = user
        self.previous_user_pool.add(user)

    def chat(self, user, text):
        self.set_new_user(user)
        message = self.get_message(user, text)
        if self.llm:
            response = self.llm(message)
            self.append_message(response)
            return self.llm(message)

    async def async_chat(self, user, text):
        self.set_new_user(user)
        message = self.get_message(user, text)
        if self.llm_async:
            response = await self.llm_async(message)
            self.append_message(response)
            return self.llm_async(message)

    def parse_rag_from_persona(self, persona, text = None):
        #每个query_rag需要饱含
        # "n" 需要几个story
        # "max_token" 最多允许多少个token，如果-1则不限制
        # "query" 需要查询的内容，如果等同于"default"则替换为text
        # "lid" 需要替换的行，这里直接进行行替换，忽视行的其他内容

        query_rags = parse_rag( persona )

        if text is not None:
            for rag in query_rags:
                if rag['query'] == "default":
                    rag['query'] = text

        return query_rags, self.token_counter(persona)

    def append_message( self, response , speaker = None ):
        if speaker is None:
            # 如果role是none，则认为是本角色{{role}}输出的句子
            self.history.append({"speaker":"{{user}}","content":response})
            # 叫speaker是为了和role进行区分
        else:
            self.history.append({"speaker":speaker,"content":response})

    def check_recompute_stories_token(self):
        return len(self.db.metas) == len(self.db.stories)

    def recompute_stories_token(self):
        self.db.metas = [self.token_counter(story) for story in self.db.stories]

    def rag_retrieve( self, query, n, max_token, avoid_ids = [] ):
        # 返回一个rag_id的列表
        query_vec = self.embedding(query)

        self.db.clean_flag()
        self.db.disable_story_with_ids( avoid_ids )

        retrieved_ids = self.db.search( query_vec, n )

        if self.check_recompute_stories_token():
            self.recompute_stories_token()

        sum_token = 0

        ans = []

        for i in range(0, len(retrieved_ids)):
            if i == 0:
                sum_token += self.db.metas[retrieved_ids[i]]
                ans.append(retrieved_ids[i])
                continue
            else:
                sum_token += self.db.metas[retrieved_ids[i]]
                if sum_token <= max_token:
                    ans.append(retrieved_ids[i])
                else:
                    break

        return ans


    def rag_retrieve_all( self, query_rags, rest_limit ):
        # 返回一个rag_ids的列表
        retrieved_ids = []
        rag_ids = []

        for query_rag in query_rags:
            query = query_rag['query']
            n = query_rag['n']
            max_token = rest_limit
            if rest_limit > query_rag['max_token'] and query_rag['max_token'] > 0:
                max_token = query_rag['max_token']

            rag_id = self.rag_retrieve( query, n, max_token, avoid_ids = retrieved_ids )
            rag_ids.append( rag_id )
            retrieved_ids += rag_id

        return rag_ids

    def append_history_under_limit(self, message, rest_limit):
        # 返回一个messages的列表
        # print("call append_history_under_limit")

        # 从后往前计算token，不超过rest_limit,
        # 如果speaker是{{role}},则message的role是assistant

        return message

    def get_message(self, user, text):
        query_token = self.token_counter(text)

        # 首先获取需要多少个rag story
        query_rags, persona_token = self.parse_rag_from_persona( self.persona, text )
        #每个query_rag需要饱含
        # "n" 需要几个story
        # "max_token" 最多允许多少个token，如果-1则不限制
        # "query" 需要查询的内容，如果等同于"default"则替换为text
        # "lid" 需要替换的行，这里直接进行行替换，忽视行的其他内容



        rest_limit = self.max_input_token - persona_token - query_token

        if self.verbose:
            print(f"query_rags: {query_rags} rest_limit = { rest_limit }")

        rag_ids = self.rag_retrieve_all( query_rags, rest_limit )

        # 将rag_ids对应的故事 替换到persona中
        augmented_persona = self.augment_persona( self.persona, rag_ids, query_rags )

        system_prompt = self.package_system_prompt( self.role_name, augmented_persona )

        token_for_system = self.token_counter( system_prompt )

        rest_limit = self.max_input_token - token_for_system - query_token

        message = [{"role":"system","content":system_prompt}]

        message = self.append_history_under_limit( message, rest_limit )

        # message.append({"role":"user","content":text})
        self.last_msg = {"role":"user","content":text}

        return message

    def package_system_prompt(self, role_name, augmented_persona):
        bot_name = role_name
        return f"""You are now in roleplay conversation mode. Pretend to be {bot_name} whose persona follows:
{augmented_persona}

You will stay in-character whenever possible, and generate responses as if you were {bot_name}"""


    def augment_persona(self, persona, rag_ids, query_rags):
        lines = persona.split("\n")
        for rag_id, query_rag in zip(rag_ids, query_rags):
            lid = query_rag['lid']
            new_text = ""
            for id in rag_id:
                new_text += "###\n" + self.db.stories[id].strip() + "\n"
            new_text = new_text.strip()
            lines[lid] = new_text
        return "\n".join(lines)

    def load_role_from_jsonl( self, role_from_jsonl ):
        if self.verbose:
            print(f"因为懒得测试，jsonl的分支还没有测试，当你使用的时候如果通过了，请告诉鲁叔测试通过了")
        import json
        datas = []
        with open(role_from_jsonl, 'r') as f:
            for line in f:
                try:
                    datas.append(json.loads(line))
                except:
                    continue

        column_name = ""

        if self.embed_name in embedname2columnname:
            column_name = embedname2columnname[self.embed_name]
        else:
            print('warning! unkown embedding name ', self.embed_name ,' while loading role')
            column_name = 'luotuo_openai'

        stories, story_vecs, persona = self.extract_text_vec_from_datas(datas, column_name)

        return persona, None, stories, story_vecs


    def load_role_from_hf(self, role_from_hf):
        # 从hf加载role
        # self.persona, new_role_name, self.stories, self.story_vecs = self.load_role_from_hf(role_from_hf)

        from datasets import load_dataset

        if role_from_hf.count("/") == 1:
            dataset = load_dataset(role_from_hf)
            datas = dataset["train"]
        elif role_from_hf.count("/") >= 2:
            split_index = role_from_hf.index('/')
            second_split_index = role_from_hf.index('/', split_index+1)
            dataset_name = role_from_hf[:second_split_index]
            split_name = role_from_hf[second_split_index+1:]

            fname = split_name + '.jsonl'
            dataset = load_dataset(dataset_name,data_files={'train':fname})
            datas = dataset["train"]

        column_name = ""

        if self.embed_name in embedname2columnname:
            column_name = embedname2columnname[self.embed_name]
        else:
            print('warning! unkown embedding name ', self.embed_name ,' while loading role')
            column_name = 'luotuo_openai'

        stories, story_vecs, persona = self.extract_text_vec_from_datas(datas, column_name)

        return persona, None, stories, story_vecs

    def extract_text_vec_from_datas(self, datas, column_name):
        # 从datas中提取text和vec
        # extract text and vec from huggingface dataset
        # return texts, vecs
        # from .utils import base64_to_float_array

        texts = []
        vecs = []
        for data in datas:
            if data[column_name] == 'system_prompt':
                system_prompt = get_text_from_data( data )
            elif data[column_name] == 'config':
                pass
            else:
                vec = base64_to_float_array( data[column_name] )
                text = get_text_from_data( data )
                vecs.append( vec )
                texts.append( text )
        return texts, vecs, system_prompt

    def load_role_from_jsonl(self, role_from_jsonl):
        # 从jsonl加载role
        return None

    def extract_story_vecs(self, stories):
        # 从stories中提取story_vecs

        if self.verbose:
            print(f"re-extract vector for {len(stories)} stories")

        from tqdm import tqdm
        story_vecs = []
        for story in tqdm(stories):
            story_vecs.append(self.embedding(story))

        return story_vecs

    def build_db(self, stories, story_vecs):
        # db的构造函数
        if self.db is None:
            self.db = NaiveDB()
        self.db.build_db(stories, story_vecs)

